In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

!pip install ultralytics -q

DRIVE_BASE = "/content/drive/MyDrive/FridgeDetection"
DATASET_DIR = f"{DRIVE_BASE}/dataset/yolo"
PROJECT_DIR = f"{DRIVE_BASE}/runs"

os.makedirs(PROJECT_DIR, exist_ok=True)

print("Setup complete.")

In [ ]:
import yaml
from pathlib import Path
from collections import defaultdict

DATA_YAML = f"{DATASET_DIR}/data.yaml"

with open(DATA_YAML, 'r') as f:
    config = yaml.safe_load(f)

config['path'] = DATASET_DIR

COLAB_YAML = "/content/data.yaml"
with open(COLAB_YAML, 'w') as f:
    yaml.dump(config, f)

print(f"Classes ({config['nc']}): {config['names']}")
print("-" * 50)

for split in ['train', 'val', 'test']:
    img_path = f"{DATASET_DIR}/images/{split}"
    lbl_path = f"{DATASET_DIR}/labels/{split}"
    if os.path.exists(img_path):
        img_count = len([f for f in os.listdir(img_path) if f.endswith(('.jpg', '.png'))])
        lbl_count = len([f for f in os.listdir(lbl_path) if f.endswith('.txt')]) if os.path.exists(lbl_path) else 0
        print(f"{split}: {img_count} images, {lbl_count} labels")

In [ ]:
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

LABEL_DIR = Path(f"{DATASET_DIR}/labels/train")

class_counts = defaultdict(int)
total_objects = 0

for label_file in tqdm(list(LABEL_DIR.glob("*.txt")), desc="Scanning labels"):
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if parts:
                class_id = int(parts[0])
                class_counts[class_id] += 1
                total_objects += 1

print(f"\nClass Distribution ({total_objects:,} total objects)")
print("-" * 40)

names = config['names']
for cid in range(len(names)):
    name = names[cid] if isinstance(names, list) else names.get(cid, f"class_{cid}")
    count = class_counts[cid]
    pct = (count / total_objects * 100) if total_objects > 0 else 0
    print(f"{cid:2d}. {name:15s}: {count:6,} ({pct:5.1f}%)")

In [ ]:
from ultralytics import YOLO
import torch

MODEL = "yolov8m-seg.pt"      # segmentation model
EPOCHS = 100
BATCH_SIZE = 16
IMG_SIZE = 640
SAVE_PERIOD = 10
PATIENCE = 50

print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Model: {MODEL} | Epochs: {EPOCHS} | Batch: {BATCH_SIZE}")
print("-" * 50)

model = YOLO(MODEL)

results = model.train(
    data=COLAB_YAML,
    project=PROJECT_DIR,
    name="fridge_yolov8m_seg",
    epochs=EPOCHS,
    patience=PATIENCE,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    save_period=SAVE_PERIOD,
    workers=4,
    optimizer="SGD",
    lr0=0.01,
    lrf=0.01,
    warmup_epochs=3,
    mosaic=1.0,
    mixup=0.1,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    plots=True,
    exist_ok=True
)

print(f"\nTraining complete. Best model: {PROJECT_DIR}/fridge_yolov8m_seg/weights/best.pt")

In [ ]:
# Resume from last checkpoint
from ultralytics import YOLO

CHECKPOINT = f"{PROJECT_DIR}/fridge_yolov8m_seg/weights/last.pt"

print(f"Resuming from: {CHECKPOINT}")

model = YOLO(CHECKPOINT)
results = model.train(resume=True)

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import random
import glob

BEST_MODEL = f"{PROJECT_DIR}/fridge_yolov8m_seg/weights/best.pt"

model = YOLO(BEST_MODEL)

val_images = glob.glob(f"{DATASET_DIR}/images/val/*.jpg") + glob.glob(f"{DATASET_DIR}/images/val/*.png")
sample_images = random.sample(val_images, min(4, len(val_images)))

fig, axes = plt.subplots(2, 2, figsize=(14, 14))

for ax, img_path in zip(axes.flat, sample_images):
    results = model.predict(img_path, conf=0.25, verbose=False)
    result_img = results[0].plot()
    ax.imshow(result_img)
    ax.set_title(f"{len(results[0].boxes)} detections")
    ax.axis('off')

plt.tight_layout()
plt.show()